In [ ]:
from google.colab import files
uploaded = files.upload()

import numpy as np
import pandas as pd
df = pd.read_csv("input_data.csv")


Saving input_data.csv to input_data.csv


Displaying dataset

In [ ]:
df.head()


,Unnamed: 0,date,item_id,batch_id,location,current_hop,external_storage,thermal_shipper_temp_reading,room_temp_reading,room_humidity_reading,item_expiry_hours,ultra_low_temperature_freezer_hours,out_of_bound_temperature_hours,refrigeration_temperature_hours
0,0,10/07/2020 14:00,vaccine_pfizerbiontech,batch001,Jharkand,dest_discarded_vaccine_storage_unit,vaccine_storage_unit,14,22,75,-1,720,1,119
1,0,10/07/2020 13:00,vaccine_pfizerbiontech,batch001,Jharkand,dest_vaccine_storage_unit,vaccine_storage_unit,4,7,93,0,720,1,119
2,0,10/07/2020 12:00,vaccine_pfizerbiontech,batch001,Jharkand,dest_vaccine_storage_unit,vaccine_storage_unit,3,6,93,1,720,1,118
3,0,10/07/2020 11:00,vaccine_pfizerbiontech,batch001,Jharkand,dest_vaccine_storage_unit,vaccine_storage_unit,3,8,93,2,720,1,117
4,0,10/07/2020 10:00,vaccine_pfizerbiontech,batch001,Jharkand,dest_vaccine_storage_unit,vaccine_storage_unit,4,6,93,3,720,1,116


**Data Cleaning **

Handling missing values



In [ ]:
print("\nMissing values in each column before cleaning:")
print(df.isnull().sum())
# Handling Missing Values

# Separate numeric and categorical columns
numeric_cols = df.select_dtypes(include=['float64', 'int64']).columns
categorical_cols = df.select_dtypes(include=['object']).columns


# Fill missing values for numeric columns with their mean
df[numeric_cols] = df[numeric_cols].fillna(df[numeric_cols].mean())

# Fill missing values for categorical columns with their mode
for col in categorical_cols:
    df[col].fillna(df[col].mode()[0], inplace=True)

# Removing Duplicates

print("\nNumber of duplicate rows:", df.duplicated().sum())
df = df.drop_duplicates()

# Display the DataFrame after cleaning
print("\nData after cleaning:")
print(df.head())

#Verify that there are no missing values left
print("\nMissing values after cleaning:")
print(df.isnull().sum())




Missing values in each column before cleaning:
Unnamed: 0                             0
date                                   0
item_id                                0
batch_id                               0
location                               0
current_hop                            0
external_storage                       0
thermal_shipper_temp_reading           0
room_temp_reading                      0
room_humidity_reading                  0
item_expiry_hours                      0
ultra_low_temperature_freezer_hours    0
out_of_bound_temperature_hours         0
refrigeration_temperature_hours        0
dtype: int64

Number of duplicate rows: 301

Data after cleaning:
   Unnamed: 0              date                 item_id  batch_id  location  \
0           0  10/07/2020 14:00  vaccine_pfizerbiontech  batch001  Jharkand   
1           0  10/07/2020 13:00  vaccine_pfizerbiontech  batch001  Jharkand   
2           0  10/07/2020 12:00  vaccine_pfizerbiontech  batch001  Jharkand

<ipython-input-3-acdc2bc09cd7>:15: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[col].fillna(df[col].mode()[0], inplace=True)


Removing Duplicates

In [ ]:
# Check for duplicates
print("Number of duplicate rows:", df.duplicated().sum())

# Remove duplicate rows if any
df = df.drop_duplicates()


Number of duplicate rows: 0


Correcting Datatypes

In [ ]:
# Print column names
print(df.columns)


Index(['Unnamed: 0', 'date', 'item_id', 'batch_id', 'location', 'current_hop',
       'external_storage', 'thermal_shipper_temp_reading', 'room_temp_reading',
       'room_humidity_reading', 'item_expiry_hours',
       'ultra_low_temperature_freezer_hours', 'out_of_bound_temperature_hours',
       'refrigeration_temperature_hours'],
      dtype='object')


In [ ]:
# Check the column names to ensure relevant columns exist
print("Column names in the DataFrame:\n", df.columns)

# Check the data types of each column
print("Data types of each column:\n", df.dtypes)

# Create a datetime column from 'year', 'month', 'day', and 'hour' if those columns exist
if {'year', 'month', 'day', 'hour'}.issubset(df.columns):
    df['datetime'] = pd.to_datetime(df[['year', 'month', 'day', 'hour']].astype(str).agg('-'.join, axis=1), errors='coerce')
else:
    print("'year', 'month', 'day', or 'hour' columns are missing. Cannot create 'datetime' column.")

# Convert numerical columns to float or integer where appropriate
numerical_columns = [
    'thermal_shipper_temp_reading',
    'room_temp_reading',
    'room_humidity_reading',
    'item_expiry_hours',
    'ultra_low_temperature_freezer_hours',
    'out_of_bound_temperature_hours',
    'refrigeration_temperature_hours'
]

for column in numerical_columns:
    if column in df.columns:
        df[column] = pd.to_numeric(df[column], errors='coerce')
    else:
        print(f"Column '{column}' not found in the DataFrame. Skipping conversion.")

# convert 'item_id', 'batch_id', 'location', and 'current_hop' columns to strings (
for column in ['item_id', 'batch_id', 'location', 'current_hop_dest_air_cargo', 'current_hop_dest_discarded_vaccine_storage_unit',
               'current_hop_dest_reefer_truck', 'current_hop_dest_vaccine_storage_unit', 'current_hop_immunization_site',
               'current_hop_last_mile', 'current_hop_source_air_cargo', 'current_hop_source_hub', 'current_hop_source_reefer truck',
               'external_storage_local_transport', 'external_storage_reefer_truck', 'external_storage_ultra_low_temperature_freezer',
               'external_storage_vacccincation_room', 'external_storage_vaccine_storage_unit']:
    if column in df.columns:
        df[column] = df[column].astype(str)
    else:
        print(f"Column '{column}' not found in the DataFrame. Skipping conversion.")

# Verify the changes by checking data types again
print("Updated Data types of each column:\n", df.dtypes)


Column names in the DataFrame:
 Index(['Unnamed: 0', 'date', 'item_id', 'batch_id', 'location', 'current_hop',
       'external_storage', 'thermal_shipper_temp_reading', 'room_temp_reading',
       'room_humidity_reading', 'item_expiry_hours',
       'ultra_low_temperature_freezer_hours', 'out_of_bound_temperature_hours',
       'refrigeration_temperature_hours'],
      dtype='object')
Data types of each column:
 Unnamed: 0                              int64
date                                   object
item_id                                object
batch_id                               object
location                               object
current_hop                            object
external_storage                       object
thermal_shipper_temp_reading            int64
room_temp_reading                       int64
room_humidity_reading                   int64
item_expiry_hours                       int64
ultra_low_temperature_freezer_hours     int64
out_of_bound_temperature_hours 

In [ ]:
print(df.dtypes)


Unnamed: 0                              int64
date                                   object
item_id                                object
batch_id                               object
location                               object
current_hop                            object
external_storage                       object
thermal_shipper_temp_reading            int64
room_temp_reading                       int64
room_humidity_reading                   int64
item_expiry_hours                       int64
ultra_low_temperature_freezer_hours     int64
out_of_bound_temperature_hours          int64
refrigeration_temperature_hours         int64
dtype: object


Outlier Detection:

In [ ]:
# Describe numerical columns to get a sense of outliers
print(df.describe())

# Select only the numerical columns
numeric_df = df.select_dtypes(include=['float64', 'int64'])

# Remove rows with outliers using IQR
Q1 = numeric_df.quantile(0.25)
Q3 = numeric_df.quantile(0.75)
IQR = Q3 - Q1

# Remove rows with outliers (those outside the 1.5*IQR range)
df_cleaned = df[~((numeric_df < (Q1 - 1.5 * IQR)) | (numeric_df > (Q3 + 1.5 * IQR))).any(axis=1)]

# Optionally, print the cleaned DataFrame and its stats
print("Cleaned DataFrame:")
print(df_cleaned.describe())


       Unnamed: 0  thermal_shipper_temp_reading  room_temp_reading  \
count     26373.0                  26373.000000       26373.000000   
mean          0.0                    -53.968263           1.023623   
std           0.0                     28.586701          21.444494   
min           0.0                    -71.000000         -71.000000   
25%           0.0                    -69.000000           6.000000   
50%           0.0                    -67.000000           7.000000   
75%           0.0                    -65.000000           8.000000   
max           0.0                     15.000000          25.000000   

       room_humidity_reading  item_expiry_hours  \
count           26373.000000       26373.000000   
mean               91.806165         575.333940   
std                 2.944769         223.165549   
min                65.000000          -1.000000   
25%                91.000000         416.000000   
50%                92.000000         628.000000   
75%         

**Encoding**

Label encoding

In [ ]:
# Identify categorical columns (object data types)
categorical_cols = df.select_dtypes(include=['object']).columns

# Display the categorical columns
print("Categorical columns:", categorical_cols)
from sklearn.preprocessing import LabelEncoder

# Initialize LabelEncoder
label_encoder = LabelEncoder()

# Loop through each categorical column and apply label encoding
for col in categorical_cols:
    # Check if the column contains the keyword for 'current_hop' or 'external_storage' or 'location'
    if 'current_hop' in col or 'external_storage' in col or 'location' in col:
        # Apply label encoding to the column
        df[col] = label_encoder.fit_transform(df[col])

# Display the DataFrame with label-encoded columns
print("\nDataFrame after label encoding:")
print(df.head())
# Verify the updated data types and first few rows
print("\nUpdated data types:")
print(df.dtypes)

# Display the first few rows of the DataFrame
print("\nFirst few rows of the DataFrame after encoding:")
print(df.head())
# Handle missing values before encoding by filling with the mode
for col in categorical_cols:
    if 'current_hop' in col or 'external_storage' in col or 'location' in col:
        df[col].fillna(df[col].mode()[0], inplace=True)

# apply label encoding to the cleaned columns
for col in categorical_cols:
    if 'current_hop' in col or 'external_storage' in col or 'location' in col:
        df[col] = label_encoder.fit_transform(df[col])



Categorical columns: Index(['date', 'item_id', 'batch_id', 'location', 'current_hop',
       'external_storage'],
      dtype='object')

DataFrame after label encoding:
   Unnamed: 0              date                 item_id  batch_id  location  \
0           0  10/07/2020 14:00  vaccine_pfizerbiontech  batch001         5   
1           0  10/07/2020 13:00  vaccine_pfizerbiontech  batch001         5   
2           0  10/07/2020 12:00  vaccine_pfizerbiontech  batch001         5   
3           0  10/07/2020 11:00  vaccine_pfizerbiontech  batch001         5   
4           0  10/07/2020 10:00  vaccine_pfizerbiontech  batch001         5   

   current_hop  external_storage  thermal_shipper_temp_reading  \
0            2                 5                            14   
1            4                 5                             4   
2            4                 5                             3   
3            4                 5                             3   
4            4            

<ipython-input-9-a92e3d7ef117>:31: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[col].fillna(df[col].mode()[0], inplace=True)
<ipython-input-9-a92e3d7ef117>:31: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using

**Feature Scaling**

Standardization:

In [ ]:
from sklearn.preprocessing import StandardScaler

# Initialize StandardScaler
scaler = StandardScaler()

# List of numerical columns to scale
numerical_cols = ['thermal_shipper_temp_reading', 'room_temp_reading', 'room_humidity_reading',
                  'item_expiry_hours', 'ultra_low_temperature_freezer_hours', 'out_of_bound_temperature_hours',
                  'refrigeration_temperature_hours']

# Apply standardization
df[numerical_cols] = scaler.fit_transform(df[numerical_cols])


Creating additional Feautures for efficient data visualisation

In [ ]:
# Check if necessary columns exist before proceeding
if {'year', 'month', 'day', 'hour'}.issubset(df.columns):
    # Create a datetime column from the 'year', 'month', 'day', and 'hour' columns
    df['datetime'] = pd.to_datetime(df[['year', 'month', 'day', 'hour']].astype(str).agg('-'.join, axis=1), errors='coerce')

    # Extract features from the datetime column
    df['year'] = df['datetime'].dt.year
    df['month'] = df['datetime'].dt.month
    df['day'] = df['datetime'].dt.day
    df['hour'] = df['datetime'].dt.hour
    df['weekday'] = df['datetime'].dt.weekday  # 0: Monday, 6: Sunday
    df['is_weekend'] = df['weekday'].apply(lambda x: 1 if x >= 5 else 0)  # 1 if weekend (Saturday/Sunday)
    df['quarter'] = df['datetime'].dt.quarter  # Quarter of the year (1-4)
    df['is_month_start'] = df['datetime'].dt.is_month_start.astype(int)  # Whether it's the start of the month
    df['is_month_end'] = df['datetime'].dt.is_month_end.astype(int)  # Whether it's the end of the month

    # Adding season based on month (Seasonal feature)
    season_map = {
        12: 'Winter', 1: 'Winter', 2: 'Winter',  # Winter: Dec, Jan, Feb
        3: 'Spring', 4: 'Spring', 5: 'Spring',  # Spring: Mar, Apr, May
        6: 'Summer', 7: 'Summer', 8: 'Summer',  # Summer: Jun, Jul, Aug
        9: 'Fall', 10: 'Fall', 11: 'Fall'  # Fall: Sep, Oct, Nov
    }
    df['season'] = df['month'].map(season_map)

    # Additional time-related features
    df['day_of_year'] = df['datetime'].dt.dayofyear  # Day of the year (1-365/366)
    df['week_of_year'] = df['datetime'].dt.isocalendar().week  # Week of the year (1-52)

    # Optional rolling features (e.g., rolling average of 'item_expiry_hours' over the last 3 days)
    df['rolling_expiry'] = df['item_expiry_hours'].rolling(window=3, min_periods=1).mean()

    # Optionally, you can drop the datetime column if it's not needed for further analysis
    df.drop(columns=['datetime'], inplace=True)

    # Verify the new features
    print("New features extracted:\n", df[['year', 'month', 'day', 'hour', 'weekday', 'is_weekend', 'quarter', 'is_month_start', 'is_month_end', 'season', 'day_of_year', 'week_of_year', 'rolling_expiry']].head())



**Data Splitting**

In [ ]:
from sklearn.model_selection import train_test_split

# Assuming 'df' is your cleaned DataFrame and you want to split the data into X (features) and y (target)
# Replace 'item_id' with the actual column you want to predict, as it seems to be the target
X = df.drop(columns=['item_id'])  # Drop target column (adjust if needed)
y = df['item_id']  # Replace 'item_id' with your actual target column

# First, split into 70% training and 30% temporary (validation + test)
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)

# Now, split the 30% temporary set into 50% validation and 50% test, resulting in 15% validation and 15% test
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# Check the sizes of each split
print(f"Training set size: {X_train.shape[0]} samples")
print(f"Validation set size: {X_val.shape[0]} samples")
print(f"Test set size: {X_test.shape[0]} samples")

# Optionally, print the first few rows of each split to verify
print("\nFirst few rows of training set:")
print(X_train.head())

print("\nFirst few rows of validation set:")
print(X_val.head())

print("\nFirst few rows of test set:")
print(X_test.head())


Training set size: 18461 samples
Validation set size: 3956 samples
Test set size: 3956 samples

First few rows of training set:
       Unnamed: 0              date  batch_id  location  current_hop  \
25743           0   07/06/2021 2:00  batch028         5            4   
14799           0  12/07/2020 11:00  batch004         1            1   
4743            0  10/30/2020 14:00  batch012         5            4   
9124            0  01/14/2021 19:00  batch021        10            4   
17532           0  09/16/2020 20:00  batch011        10            4   

       external_storage  thermal_shipper_temp_reading  room_temp_reading  \
25743                 5                     -0.560822           0.325329   
14799                 5                      2.132790           0.325329   
4743                  5                     -0.455876           0.232063   
9124                  5                     -0.455876           0.232063   
17532                 5                     -0.420894      

Performing Data Aggregation

In [ ]:
# Ensure that the 'date' column is in datetime format
df['date'] = pd.to_datetime(df['date'], errors='coerce')

# Extract month, hour, and year from the 'date' column
df['month'] = df['date'].dt.month  # Extract month
df['hour'] = df['date'].dt.hour    # Extract hour
df['year'] = df['date'].dt.year    # Extract year

# Now, proceed with the rest of the aggregation code
# Data Aggregation: Group by relevant columns and aggregate the data

# Group by 'location' columns and calculate the mean of numerical columns
location_columns = ['location']  # List of location columns
location_agg = df.groupby(location_columns)[numerical_cols].agg(['mean', 'sum', 'min', 'max'])

# Aggregating by 'current_hop' (updated from 'current_hop_dest_air_cargo') and calculating statistics on numerical columns
current_hop_agg = df.groupby('current_hop')[numerical_cols].agg(['mean', 'std', 'sum'])

# Aggregating by 'month' to see monthly patterns for numerical columns
monthly_agg = df.groupby('month')[numerical_cols].agg(['mean', 'sum', 'std'])

# Aggregating by 'hour' to observe hourly variations
hourly_agg = df.groupby('hour')[numerical_cols].agg(['mean', 'sum', 'min', 'max'])

# Aggregate based on 'item_id', which might help understand patterns per item
item_agg = df.groupby('item_id')[numerical_cols].agg(['mean', 'std', 'sum'])

# Aggregating using 'year' to summarize yearly trends
yearly_agg = df.groupby('year')[numerical_cols].agg(['mean', 'sum', 'min', 'max'])

# Counting the occurrences of each location
location_count_agg = df[location_columns].notnull().sum()

# Optionally, reset the index to turn groupby indices into columns
location_agg = location_agg.reset_index()
current_hop_agg = current_hop_agg.reset_index()
monthly_agg = monthly_agg.reset_index()
hourly_agg = hourly_agg.reset_index()
item_agg = item_agg.reset_index()
yearly_agg = yearly_agg.reset_index()

# Display some of the aggregated results
print("\nAggregated data (Location):")
print(location_agg.head())

print("\nAggregated data (Current Hop):")
print(current_hop_agg.head())

print("\nAggregated data (Monthly):")
print(monthly_agg.head())

print("\nAggregated data (Hourly):")
print(hourly_agg.head())

print("\nAggregated data (Item ID):")
print(item_agg.head())

print("\nAggregated data (Yearly):")
print(yearly_agg.head())

print("\nCount of occurrences per Location:")
print(location_count_agg)




Aggregated data (Location):
  location thermal_shipper_temp_reading                                   \
                                   mean          sum       min       max   
0        0                     0.913418  1195.664528 -0.560822  2.377663   
1        1                     1.766547  2893.603680 -0.595804  2.412645   
2        2                     1.533410  1327.933417 -0.560822  2.377663   
3        3                    -0.525175  -165.955451 -0.560822 -0.490858   
4        4                    -0.561428   -97.127092 -0.595804 -0.525840   

  room_temp_reading                                 room_humidity_reading  \
               mean         sum       min       max                  mean   
0          0.282187  369.382848  0.232063  1.071455             -0.653311   
1         -0.091549 -149.957136 -3.358670  1.118088             -1.076928   
2          0.396947  343.756428  0.232063  1.118088             -1.390969   
3          0.839766  265.366169  0.791658  0.884924  

In [ ]:
df.columns

Index(['Unnamed: 0', 'date', 'item_id', 'batch_id', 'location', 'current_hop',
       'external_storage', 'thermal_shipper_temp_reading', 'room_temp_reading',
       'room_humidity_reading', 'item_expiry_hours',
       'ultra_low_temperature_freezer_hours', 'out_of_bound_temperature_hours',
       'refrigeration_temperature_hours', 'month', 'hour', 'year'],
      dtype='object')

In [ ]:
df.to_csv('final_data.csv',index='False')